In [2]:
import pandas as pd
import numpy as np
import re
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from keras.models import Sequential
from keras.layers import Dense
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
from gettext import install


install tensorflow
import os


SyntaxError: invalid syntax (2762561243.py, line 1)

In [ ]:
# Load dataset (assumes CSV with columns like 'email_body' and 'label')
df = pd.read_csv("emails.csv")

# Sample preprocessing function
def preprocess(text):
    text = re.sub(r'<[^>]+>', '', text)
    text = re.sub(r'\W+', ' ', text)
    text = text.lower()
    tokens = text.split()
    tokens = [t for t in tokens if t not in stopwords.words('english')]
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(t) for t in tokens]
    return ' '.join(tokens)

df['clean_text'] = df['email_body'].apply(preprocess)

In [ ]:
# TF-IDF Vectorization
vectorizer = TfidfVectorizer(max_features=3000)
X = vectorizer.fit_transform(df['clean_text']).toarray()
y = df['label']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Random Forest
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
print("Random Forest Results:\n", classification_report(y_test, y_pred_rf))

In [ ]:
pip install keras.models


In [ ]:
# SVM
svm = SVC(kernel='linear')
svm.fit(X_train, y_train)
y_pred_svm = svm.predict(X_test)
print("SVM Results:\n", classification_report(y_test, y_pred_svm))

In [ ]:
# ANN
ann = Sequential()
ann.add(Dense(512, input_dim=X_train.shape[1], activation='relu'))
ann.add(Dense(256, activation='relu'))
ann.add(Dense(1, activation='sigmoid'))
ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
ann.fit(X_train, y_train, epochs=10, batch_size=64, verbose=1)
y_pred_ann = (ann.predict(X_test) > 0.5).astype("int32")
print("ANN Results:\n", classification_report(y_test, y_pred_ann))

In [ ]:
# NER with spaCy
import spacy
nlp = spacy.load("en_core_web_sm")

def extract_entities(text):
    doc = nlp(text)
    return [(ent.text, ent.label_) for ent in doc.ents]

# Example
sample_email = df['email_body'].iloc[0]
entities = extract_entities(sample_email)
print("Named Entities:", entities)

In [ ]:
# Save models
import joblib
joblib.dump(rf, "random_forest_model.pkl")
joblib.dump(svm, "svm_model.pkl")
ann.save("ann_model.h5")